<a href="https://colab.research.google.com/github/POmegrenadier9090/keyboard_arr_optimizer/blob/main/Keyboard_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
from google.colab import drive

drive.mount('/content/drive')
%cd "/content/drive/My Drive/Project"
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Project
gooLab_hiragana.txt  Keyboard_Optimizer.ipynb  sample_text_営業意思決定支援.txt


In [96]:
from sys import setswitchinterval
import numpy as np
import pandas as pd
import math
import requests
import chardet

pd.set_option('display.max_columns', None)

alphabet_index = {
    'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6,
    'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12,
    'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18,
    't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25
}

index_alphabet = {
  0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g',
  7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm',
  13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's',
  19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z'
}

key_cost = np.array(
        [
            [22,18,10,9,15,15,9,10,18,22],
            [11,7, 1, 1,7, 7, 1,1, 7, 11],
            [22,18,10,9,15,15,9,10,18,22]
    ]
        )

#calculates the coordinates difining "z" as the origin
key_position = np.array(
    [#abcdefghijklmf
     [0,1],[4,0],[2,0],[2,1],[2,2],[3,1],[4,1],[5,1],[7,2],[6,1],[7,1],[8,1],[6,0],
     #nopqrstuvwxyz
     [5,0],[8,2],[9,2],[0,2],[3,2],[1,1],[4,2],[6,2],[3,0],[1,2],[1,0],[5,2],[0,0]
     ]
)
print(key_position[20])
# ひらがな配列をローマ字に変換する関数
def convert_to_romaji(hiragana_string):
  hiragana_to_romaji = {
    "あ": "a", "い": "i", "う": "u", "え": "e", "お": "o",
    "か": "ka", "き": "ki", "く": "ku", "け": "ke", "こ": "ko",
    "さ": "sa", "し": "si", "す": "su", "せ": "se", "そ": "so",
    "た": "ta", "ち": "ti", "つ": "tu", "て": "te", "と": "to",
    "な": "na", "に": "ni", "ぬ": "nu", "ね": "ne", "の": "no",
    "は": "ha", "ひ": "hi", "ふ": "fu", "へ": "he", "ほ": "ho",
    "ま": "ma", "み": "mi", "む": "mu", "め": "me", "も": "mo",
    "や": "ya",             "ゆ": "yu",             "よ": "yo",
    "ら": "ra", "り": "ri", "る": "ru", "れ": "re", "ろ": "ro",
    "わ": "wa",                                     "を": "wo", "ん": "nn",
    "が": "ga", "ぎ": "gi", "ぐ": "gu", "げ": "ge", "ご": "go",
    "ざ": "za", "じ": "ji", "ず": "zu", "ぜ": "ze", "ぞ": "zo",
    "だ": "da", "ぢ": "ji", "づ": "zu", "で": "de", "ど": "do",
    "ば": "ba", "び": "bi", "ぶ": "bu", "べ": "be", "ぼ": "bo",
    "ぱ": "pa", "ぴ": "pi", "ぷ": "pu", "ぺ": "pe", "ぽ": "po",
    "ゔ": "vu",
    "ぁ": "la", "ぃ": "li", "ぅ": "lu", "ぇ":"le", "ぉ":"lo"
  }
  lower_vowel = {"ぁ": "a", "ぃ": "i", "ぅ": "u", "ぇ":"e", "ぉ":"o"}
  y_column_vowel = {"ゃ":"a", "ゅ": "u", "ょ": "o"}

  def two_letter_input(cur_char, next_char,flg):
    if cur_char in ("ふ", "ゔ") and next_char in lower_vowel:
      con = hiragana_to_romaji.get(cur_char,"")[0]
      vowel = lower_vowel.get(next_char,"")
      temp = con + vowel
      flg += 1
    elif next_char in ("ゃ","ゅ","ょ"):
      con = hiragana_to_romaji.get(cur_char,"")[0]
      vowel = y_column_vowel.get(next_char,"")
      temp = con + "y" + vowel
      flg += 1
    else:
      temp=""
      flg=0
    return temp, flg

  def lower_case_tu(cur_char, next_char, next_next_char,flg):
    con = hiragana_to_romaji.get(next_char,"")[0]
    if next_next_char in lower_vowel or next_next_char in y_column_vowel:
      temp, flg = two_letter_input(next_char, next_next_char, flg)
      temp = con + temp
      flg += 1
    else:
      vowel = hiragana_to_romaji.get(next_char,"")[1]
      temp = con *2 + vowel
      flg += 1
    return temp, flg

  romaji_result = ""
  i = 0
  r=-1
  while i < len(hiragana_string):
    if i==r:
      print("@@@@@")
      break
    r= i

    flg = 0
    cur_char = hiragana_string[i]
    if i < len(hiragana_string)-1:
      next_char = hiragana_string[i+1]
    else:
      next_char =""
    if i < len(hiragana_string)-2:
      next_next_char = hiragana_string[i+2]
    else:
      next_next_char =""
    # 分岐スタート
    #小さいっの場合
    if cur_char == "っ":
        # 次の文字が存在する場合、その文字の先頭を重ねる
      if next_char:
        temp, flg = lower_case_tu(cur_char, next_char,next_next_char,flg)
        romaji_result += temp
        flg += 1
    #2-letterの場合
    elif next_char in lower_vowel or next_char in y_column_vowel:
      temp, flg = two_letter_input(cur_char, next_char,flg)
      if not temp:
        con =hiragana_to_romaji.get(cur_char)
        lower = hiragana_to_romaji.get(next_char)
        temp = con+ lower
        flg+=1
      romaji_result += temp
      flg += 1
    #1対1変換
    else:
      romaji_result += hiragana_to_romaji.get(cur_char,"")
      flg += 1
    i += flg
  return romaji_result

def cost(prev_key, key):
  #連続して同じ手を用いるか？ 左手: 0 、右手: 1
  cost_additional = 0
  prev_hand = 0 if prev_key[0] <= 4 else 1
  cur_hand = 0 if key[0] <= 4 else 1
  #全く同じなら追加コストなし
  if prev_hand == cur_hand and not np.all(prev_key == key):
    cost_additional += 10
      #同じ側の同じ指を連続して使うならさらにコスト増加
    if (prev_key[0] in (3,4) and key[0] in (3,4)) or (prev_key[0] in (5,6) and key[0] in (5,6)):
      cost_additional += 6
    if prev_key[0] == key[0]:
      cost_additional += 6
  return key_cost[key[1],key[0]] + cost_additional

#key: 入力したキーの座標
#to be done: 上下動作の非対称性、2個飛ばしが起きた際のコスト増加、右手はEnterを使うのでコストが大きめ
def calc_avg_cost(input_list, key_position):
  total_cost = 0
  prev_key =key_position[alphabet_index.get(input_list[0])]
  for letter in input_list[1:]:
    key = key_position[alphabet_index.get(letter)]
    total_cost += cost(prev_key, key)
  return total_cost/len(input_list)

def calc_transition_N(text):
  transition_N = np.zeros((26,26))
  prev_index = alphabet_index.get(text[0])
  for letter in text[1:]:
    index = alphabet_index.get(letter)
    transition_N[prev_index, index] += 1
    prev_index = index
  return transition_N

def calc_cost_matrix(transition_N, key_position):
  N = np.sum(transition_N)
  cost_matrix = np.zeros((26,26))
  for i in range(cost_matrix.shape[0]):
    prev_key = key_position[i]
    for j in range(cost_matrix.shape[1]):
      key = key_position[j]
      cost_matrix[i,j] = math.pow(cost(prev_key, key), 2) * math.log(transition_N[i,j],7) / N if transition_N[i,j] !=0 else -0.0001
  return cost_matrix

def left_or_right(key_position):
  #各アルファベットが右か左か
  array_right = []
  array_left  = []
  for i in range(key_position.shape[0]):
    position = key_position[i,:]
    letter = index_alphabet.get(i)
    if position[0] > 4:
      array_right.append(letter)
    else:
      array_left.append(letter)

  return array_left, array_right

def load_txt(txt_file):
  with open(txt_file, 'r', encoding='UTF-8',errors = "replace") as file:

      sentence = file.read()
  #print(sentence)
  api_key = 'c7bb63f0be4f3f6f9aa980e67151d12e06dfc6f81937d2e9a244bc8e9d1f6f53'
  #sentence = '皆さんの誤解を解く目的で解説する。あっちゃーヴぁー'
  url = "https://labs.goo.ne.jp/api/hiragana"
  params = {
      "app_id": api_key,
      "sentence": sentence,
      "output_type": "hiragana"
  }
  try:
    response = requests.post(url, json = params)
    if response.status_code == 200:
      sentence_hiragana = response.json()
    else:
      print(f"error: {response.status_code}, message: {response.text}")
  except requests.exceptions.RequestException as e:
      # リクエスト中にエラーが発生した場合
      print("An error has occurred", str(e))
  converted_txt = sentence_hiragana["converted"]
  #print(converted_txt)
  romaji_txt = convert_to_romaji(converted_txt)
  print(romaji_txt)
  input_list = list(romaji_txt)
  input = np.array(input_list)
  return romaji_txt, input_list


def optimize_for_candidates(key_position, candidates_letter_list, cur_avg_cost,input_list):
  temp_key_position = key_position
  new_cost = 9999999999
  flg = 0
  for candidate in candidates_letter_list:
    #アルファベットのインデックス
    candidate_index = alphabet_index.get(candidate)
    temp = key_position[target_index[1]]
    temp_key_position[target_index[1]] = key_position[candidate_index]
    temp_key_position[candidate_index] = temp
    temp_cost = calc_avg_cost(input_list, temp_key_position)
    if temp_cost < cur_avg_cost:
      cur_avg_cost = calc_avg_cost(input_list, temp_key_position)
      new_key_position = temp_key_position
      flg = 1
  if flg == 1:
    print("improved")
  return new_key_position, cur_avg_cost

[6 2]


In [94]:
romaji_txt, input_list =load_txt('sample_text_営業意思決定支援.txt')
#transition_Nはkey_positionに依存しないため、最適化最中に更新する必要ない
transition_N = calc_transition_N(romaji_txt)  #df = pd.DataFrame(transition_N,alphabet_index.keys(), alphabet_index.keys()) display(df)
cur_avg_cost = calc_avg_cost(input_list, key_position)
print(cur_avg_cost)

...
14.351710838607595


In [87]:
#コスト寄与率を計算
df_cost_contribution = pd.DataFrame(calc_cost_matrix(transition_N, key_position),alphabet_index.keys(), alphabet_index.keys())

array_left, array_right = left_or_right(key_position)

#1:最大値とその位置を取得
#max_value = df_x.values.max()
target_letter = df_cost_contribution.stack().idxmax()
print(target_letter)
#変えるアルファベットとそのもとになるアルファベットを保存
target_index = list(map(lambda x:alphabet_index.get(x), target_letter))
print(f" アルファベットのインデックス: {target_index}")
target_letter_position = key_position[target_index[1],:]
#2:
cost_for_each_key = df_cost_contribution.sum().to_frame(name = "Sum")
candidates = cost_for_each_key.loc[array_left].sort_values('Sum',ascending=True) if target_letter_position[0] > 4 else cost_for_each_key.loc[array_right].sort_values('Sum',ascending = True)
#3: 候補の中でコストを最小化する置換先を求めて交換したnew_key_positionを求める
#候補となるkeyのリスト（アルファベット）
candidates_letter_list = candidates.index.tolist()
new_key_position = optimize_for_candidates(key_position,candidates_letter_list,cur_avg_cost, input_list)



display(cost_for_each_key)
display(candidates)
display(df_cost_contribution)
#⇒uとnの位置を変更したい。
#まず、どのキーが次に続くキーがどちら側に来やすいかを求めて初期の配列を構成すべきか？
#目指す方向としては、同じ側で連続して打つ頻度を減らすこと（こちらは上記の初期化でうまくいくかもしれない、頻度をそのまま使うことはできないので、割合で計算する必要がある。）
#打つ頻度の大きなキーはできるだけホームポジションに位置させること（初期化した中で頻度の大きいものをホームポジションに配置する。さらに、同じ手の中で連続）

#1:コスト行列内から最も値の大きな(選択方法は要検討)要素targetを選ぶ
#2:コスト行列のcolumn和を取りその中で最も値(合計寄与率)が小さな文字replacedを取得:N=0の値は-0.001ぐらいが妥当か,この際、targetのrowの文字と異なる側から選択する(逆サイドの文字全てについてシミュレーションを行い、コスト関数を最も減少させる組み合わせを求めてもいいかも)
#targetのcolumnの文字とreplacedの位置を交換する(key_position)の更新
#新しくコスト行列を計算
#εGREEDYを使うのもありかも、(確率εでランダムに位置変更)次第にεの値を縮小していくー同じキーの交換が続いてしまうかも

('u', 'n')
 アルファベットのインデックス: [20, 13]
improved


,Sum
a,0.549588
b,0.137280
c,-0.002600
d,0.016954
e,0.471023
f,0.008725
g,0.065758
h,0.170471
i,0.530973
j,0.096693


,Sum
c,-0.002600
q,-0.002600
v,-0.002600
x,-0.002600
f,0.008725
d,0.016954
g,0.065758
s,0.099960
r,0.127827
b,0.137280


,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
a,0.010246,0.047579,-0.0001,0.008327,0.038693,0.004263,0.029945,0.004104,0.014310,0.000073,0.000147,-0.000100,0.007593,0.033654,0.013251,0.036214,-0.0001,0.048372,0.034520,0.081754,0.004523,-0.0001,0.064910,-0.0001,0.013710,0.084492
b,0.036512,-0.000100,-0.0001,-0.000100,0.022334,-0.000100,-0.000100,-0.000100,0.009616,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,0.014752,-0.000100,-0.0001,-0.000100,-0.000100,-0.000100,0.007789,-0.0001,-0.000100,-0.0001,0.000000,-0.000100
c,-0.000100,-0.000100,-0.0001,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,-0.0001,-0.000100,-0.000100,-0.000100,-0.000100,-0.0001,-0.000100,-0.0001,-0.000100,-0.000100
d,0.039843,-0.000100,-0.0001,-0.000100,0.092017,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,0.039620,-0.000100,-0.0001,-0.000100,-0.000100,-0.000100,-0.000100,-0.0001,-0.000100,-0.0001,-0.000100,-0.000100
e,0.039518,0.017448,-0.0001,0.010181,0.008375,0.006394,0.018249,0.004624,0.014010,0.000081,0.000117,0.006566,0.009169,0.031570,0.014752,0.032458,-0.0001,0.048786,0.037586,0.079042,-0.000100,-0.0001,0.027619,-0.0001,0.003963,0.028587
f,-0.000100,-0.000100,-0.0001,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,0.004090,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,0.000000,-0.000100,-0.0001,-0.000100,-0.000100,-0.000100,0.007487,-0.0001,-0.000100,-0.0001,-0.000100,-0.000100
g,0.057288,-0.000100,-0.0001,-0.000100,0.032304,-0.000100,-0.000100,-0.000100,0.006706,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,0.026825,-0.000100,-0.0001,-0.000100,-0.000100,-0.000100,0.006626,-0.0001,-0.000100,-0.0001,0.023867,-0.000100
h,0.015281,-0.000100,-0.0001,-0.000100,0.007345,-0.000100,-0.000100,-0.000100,0.029929,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,0.089648,-0.000100,-0.0001,-0.000100,-0.000100,-0.000100,-0.000100,-0.0001,-0.000100,-0.0001,0.111980,-0.000100
i,0.013275,0.021233,-0.0001,0.000100,0.012270,0.000035,0.005940,0.030309,0.010903,0.010458,0.039234,0.020807,0.034504,0.087883,0.088510,0.114027,-0.0001,0.008774,0.006766,0.035068,0.019076,-0.0001,0.033710,-0.0001,0.070181,0.050560
j,-0.000100,-0.000100,-0.0001,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,0.045158,0.000000,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,-0.000100,-0.0001,-0.000100,-0.000100,-0.000100,-0.000100,-0.0001,-0.000100,-0.0001,0.114340,-0.000100
